In [2]:
#NOW WE WANT TO ANALYZE THE CLEANED DATA AND CALCULATE THE FINANCIAL METRICS 
import pandas as pd


In [3]:
df = pd.read_csv("Personal_Finance_Dataset_CLEANED.csv")


In [4]:
df["Date"] = pd.to_datetime(df["Date"])


In [5]:
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year


In [6]:
df["Amount"] = df.apply(
    lambda row: -abs(row["Amount"]) if row["Type"] == "Expense" else abs(row["Amount"]),
    axis=1
)


In [7]:
#HOW MUCH MONEY IS SPENDED EACH YEAR?
monthly_spending = (
    df[df["Type"] == "Expense"]
    .groupby(["Year", "Month"])["Amount"]
    .sum()
    .abs()
)

monthly_spending


Series([], Name: Amount, dtype: float64)

In [8]:
#where does my money go?
category_totals = (
    df[df["Type"] == "Expense"]
    .groupby("Category")["Amount"]
    .sum()
    .abs()
    .sort_values(ascending=False)
)

category_totals


Series([], Name: Amount, dtype: float64)

In [9]:
#did I earn more or spend more?
income_vs_expense = df.groupby("Type")["Amount"].sum()
income_vs_expense


Type
expense    1227194.37
income      664589.27
Name: Amount, dtype: float64

In [10]:
average_monthly_spending = monthly_spending.mean()
average_monthly_spending


nan

In [13]:
# what percentage of my income did I save?

total_income = df[df["Type"] == "Income"]["Amount"].sum()
total_expense = abs(df[df["Type"] == "Expense"]["Amount"].sum())

if total_income == 0:
    savings_rate = 0
    print("No income records found, savings rate set to 0.")
else:
    savings_rate = (total_income - total_expense) / total_income * 100

savings_rate

No income records found, savings rate set to 0.


0

In [14]:
monthly_spending.head()
category_totals.head()

Series([], Name: Amount, dtype: float64)

In [16]:
#now B2, B2 is the analysis of the numbers already calculated in B1
df["Type"].value_counts()

Type
expense    1222
income      278
Name: count, dtype: int64

In [19]:
#get the highest spending category
highest_spending_categories = (
    df[df["Type"].str.strip().str.lower() == "expense"]
    .groupby("Category")["Amount"]
    .sum()
    .abs()
    .sort_values(ascending=False)
)

highest_spending_categories

Category
travel              169497.79
rent                162075.39
food & drink        159493.39
salary              149053.55
entertainment       148165.47
shopping            146880.75
utilities           146833.97
health & fitness    145194.06
Name: Amount, dtype: float64

In [20]:
#find expenses that occurs regularly over many months
recurring_expenses = (
    df[df["Type"].str.strip().str.lower() == "expense"]
    .groupby("Category")
    .agg(
        total_spent=("Amount", lambda x: abs(x.sum())),
        months_active=("Month", "nunique")
    )
    .sort_values("months_active", ascending=False)
)

recurring_expenses

,total_spent,months_active
Category,,
entertainment,148165.47,12
food & drink,159493.39,12
health & fitness,145194.06,12
rent,162075.39,12
salary,149053.55,12
shopping,146880.75,12
travel,169497.79,12
utilities,146833.97,12


In [21]:
#now we should identify the over spending months 
monthly_spending = (
    df[df["Type"].str.strip().str.lower() == "expense"]
    .groupby(["Year", "Month"])["Amount"]
    .sum()
    .abs()
)

overspending_months = monthly_spending[
    monthly_spending > monthly_spending.mean()
]

overspending_months

Year  Month
2020  6        20680.72
      7        24026.27
      8        31386.80
      9        21515.15
      10       22162.99
      11       26735.16
      12       28358.12
2021  3        26223.24
      4        23097.71
      5        23780.78
      6        22137.63
2022  1        20832.61
      3        27736.19
      4        21445.01
      5        31344.05
      6        25557.78
      8        24246.00
2023  1        28714.75
      7        26122.96
      9        25849.78
      11       22832.52
      12       20654.33
2024  1        29916.13
      2        23807.30
      6        22554.07
      7        27716.08
      8        22933.69
      10       22984.21
      11       23565.96
Name: Amount, dtype: float64

In [22]:
#compare spending trends over time
yearly_spending_trend = (
    df[df["Type"].str.strip().str.lower() == "expense"]
    .groupby("Year")["Amount"]
    .sum()
    .abs()
)

yearly_spending_trend

Year
2020    255772.86
2021    218644.46
2022    256776.35
2023    244498.24
2024    251502.46
Name: Amount, dtype: float64